In [39]:
import math, time, random, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import pickle
import warnings
warnings.filterwarnings('ignore')

In [40]:
train = pd.read_csv("UNSW_NB15_training-set.csv")
test = pd.read_csv("UNSW_NB15_testing-set.csv")

In [41]:
train.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,0,600000000.0,0.0,0,0,0.006,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,0,850400000.0,0.0,0,0,0.010,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0


In [42]:
test.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0


In [43]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda

data = pd.concat([train,test]).reset_index(drop = True)
cols_cat = data.select_dtypes("object").columns
cols_numeric = data._get_numeric_data().columns

In [44]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data["service"].unique() # get rid of "-"
data["service"] = np.where(data["service"]=='-', "None", data["service"])
print(data["service"].unique())

['None' 'http' 'ftp' 'ftp-data' 'smtp' 'pop3' 'dns' 'snmp' 'ssl' 'dhcp'
 'irc' 'radius' 'ssh']


In [45]:
data = pd.get_dummies(data, columns = ["proto", "service", "state", "attack_cat"], dtype = "float64") 

In [46]:
data.drop(["attack_cat_Analysis", "attack_cat_Backdoor", "attack_cat_Exploits",
            "attack_cat_Fuzzers", "attack_cat_Reconnaissance", "attack_cat_Shellcode",
            "attack_cat_Worms", "attack_cat_Generic"], axis = 1, inplace = True)

In [47]:
data.head()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,...,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_None,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no,attack_cat_DoS,attack_cat_Normal
0,1,0.000011,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,0.000008,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,0.000005,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,0.000006,2,0,900,0,166666.6608,254,0,600000000.0,0.0,0,0,0.006,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0.000010,2,0,2126,0,100000.0025,254,0,850400000.0,0.0,0,0,0.010,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [48]:
data_drop = data.drop(["id", "attack_cat_DoS", "attack_cat_Normal"], axis = 1)
cols_numeric = list(cols_numeric)
cols_numeric.remove("label")
cols_numeric.remove("id")

In [49]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
data_drop[cols_numeric] = (data_drop[cols_numeric] - np.min(data_drop[cols_numeric])) / np.std(data_drop[cols_numeric])

In [50]:
data_drop.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,...,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_None,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
0,1.841222e-06,0.007356,0.0,0.002716,0.0,0.566962,2.478337,0.0,0.971102,0.0,0.0,0.0,1.589104e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.087903,0.0,0.0,0.0,0.092339,2.015524,0.000000,0.000000,0.0,0.089922,0.0,0.0,0.0,0.000000,0.091956,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.339070e-06,0.007356,0.0,0.010002,0.0,0.779572,2.478337,0.0,4.743422,0.0,0.0,0.0,1.155712e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.162200,0.0,0.0,0.0,0.092339,2.015524,0.000000,0.000000,0.0,0.089922,0.0,0.0,0.0,0.000000,0.091956,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8.369190e-07,0.007356,0.0,0.006008,0.0,1.247316,2.478337,0.0,4.600204,0.0,0.0,0.0,7.223200e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.476922,0.0,0.0,0.0,0.184678,2.015524,0.000000,0.000000,0.0,0.179844,0.0,0.0,0.0,0.000000,0.183913,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.004303e-06,0.007356,0.0,0.005041,0.0,1.039430,2.478337,0.0,3.230480,0.0,0.0,0.0,8.667840e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.068958,0.0,0.0,0.0,0.184678,2.015524,0.122343,0.122537,0.0,0.179844,0.0,0.0,0.0,0.119101,0.183913,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.673838e-06,0.007356,0.0,0.012096,0.0,0.623658,2.478337,0.0,4.578667,0.0,0.0,0.0,1.444640e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.046121,0.0,0.0,0.0,0.184678,2.015524,0.122343,0.122537,0.0,0.179844,0.0,0.0,0.0,0.119101,0.183913,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [51]:
data_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257673 entries, 0 to 257672
Columns: 197 entries, dur to state_no
dtypes: float64(196), int64(1)
memory usage: 387.3 MB


Models

In [52]:
from sklearn import model_selection
from sklearn import metrics
X = data_drop.drop("label", axis = 1)
Y = data_drop["label"]

In [53]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
def fit_algo(algo, x, y, cv):
    model = algo.fit(x, y)
    
    #check its score
    acc = round(model.score(x, y) *100, 2)
    y_pred = model_selection.cross_val_predict(algo, x, y, cv = cv,
                                              n_jobs = -1)
    
    acc_cv = round(metrics.accuracy_score(Y,y_pred)*100,2)
    
    return y_pred, acc, acc_cv, model

In [54]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.linear_model import LogisticRegression
start_time = time.time()

pred_now, acc_lr, acc_cv_lr, lr = fit_algo(LogisticRegression(C = 0.1), X, Y, 10)

lr_time = (time.time() - start_time)

print("Accuracy: %s" % acc_lr)
print("Accuracy of CV: %s" % acc_cv_lr)
print("Execution time: %s" % lr_time)

Accuracy: 90.1
Accuracy of CV: 89.04
Execution time: 100.48232007026672


In [55]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.neighbors import KNeighborsClassifier
start_time = time.time()
pred_now, acc_knn, acc_cv_knn, knn = fit_algo(KNeighborsClassifier(n_neighbors = 3)
                                        , X, Y, 10)
knn_time = (time.time() - start_time)
print("Accuracy: %s" % acc_knn)
print("Accuracy of CV: %s" % acc_cv_knn)
print("Execution time: %s" % knn_time)

Accuracy: 95.9
Accuracy of CV: 89.71
Execution time: 4167.896835327148


In [56]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
start_time = time.time()
pred_now, acc_dt, acc_cv_dt, dt = fit_algo(DecisionTreeClassifier(random_state = 1)
                                        , X, Y, 10)

dt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_dt)
print("Accuracy of CV: %s" % acc_cv_dt)
print("Execution time: %s" % dt_time)




Accuracy: 99.74
Accuracy of CV: 91.44
Execution time: 59.89781713485718


In [57]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Random Forest
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()
pred_now, acc_rf, acc_cv_rf, rf = fit_algo(RandomForestClassifier(n_estimators = 100)
                                        , X, Y, 10)

rf_time = (time.time() - start_time)

print("Accuracy: %s" % acc_rf)
print("Accuracy of CV: %s" % acc_cv_rf)
print("Execution time: %s" % rf_time)

Accuracy: 99.74
Accuracy of CV: 92.6
Execution time: 438.80466532707214


In [58]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
#Gaussian Naive Bayes 
from sklearn.naive_bayes import GaussianNB
start_time = time.time()

pred_now, acc_gnb, acc_cv_gnb, gnb= fit_algo(GaussianNB()
                                        ,X,Y,5)

gnb_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gnb)
print("Accuracy of CV: %s" % acc_cv_gnb)
print("Execution time: %s" % gnb_time)

Accuracy: 50.5
Accuracy of CV: 50.46
Execution time: 7.4349143505096436


In [59]:
 #Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
 # Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
start_time = time.time()

pred_now, acc_gbt, acc_cv_gbt, gbt= fit_algo(GradientBoostingClassifier()
                                        , X, Y, 10)

gbt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gbt)
print("Accuracy of CV: %s" % acc_cv_gbt)
print("Execution time: %s" % gbt_time)



Accuracy: 93.38
Accuracy of CV: 92.1
Execution time: 1841.9599554538727


In [60]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Cross Validation Accuracy Comparison
algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'CV Accuracy %' : [acc_cv_lr, acc_cv_dt, acc_cv_rf, acc_cv_gnb, acc_cv_gbt, acc_cv_knn] })
acc_df = acc_df.sort_values(by='CV Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df



,Algorithm,CV Accuracy %
0,RandomForest,92.60
1,GBC,92.10
2,Decision Tree,91.44
3,KNN,89.71
4,Log. Reg.,89.04
5,Gaussian NB,50.46


In [61]:
# Accuracy Comparison

algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'Accuracy %' : [acc_lr, acc_dt, acc_rf, acc_gnb, acc_gbt, acc_knn] })
acc_df = acc_df.sort_values(by='Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df

,Algorithm,Accuracy %
0,Decision Tree,99.74
1,RandomForest,99.74
2,KNN,95.90
3,GBC,93.38
4,Log. Reg.,90.10
5,Gaussian NB,50.50


In [63]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
fimp_rf = pd.DataFrame({'Feature' : X.columns, 'Importance' : (rf.feature_importances_).astype(float)})
fimp_rf = fimp_rf.sort_values(by='Importance', ascending=False)
fimp_rf.head(37)

,Feature,Importance
6,sttl,0.085595
28,ct_state_ttl,0.068380
3,sbytes,0.043514
23,smean,0.040248
22,ackdat,0.039816
32,ct_dst_src_ltm,0.039101
5,rate,0.038771
190,state_INT,0.038734
7,dttl,0.038217
8,sload,0.037963


50%

In [64]:
x = data_drop.copy()

In [65]:
from sklearn.model_selection import train_test_split
SEED = 1

x_train, x_test = train_test_split(x, train_size = 0.50, random_state = SEED)

In [66]:
print(x_train.shape)
x_train.head()

(128836, 197)


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,...,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_None,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
115986,1.667143e-04,0.007356,0.017859,0.000702,0.001218,0.018785,0.302474,0.257179,0.003157,0.296331,0.00000,0.0,1.444640e-06,0.000005,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.237979,0.350338,0.0,0.0,0.000000,0.000000,0.122343,0.000000,0.000000,0.089922,0.0,0.0,0.0,0.476403,0.183913,0.000000,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201332,6.695352e-07,0.007356,0.000000,0.000518,0.000000,1.559145,2.478337,0.000000,0.613791,0.000000,0.00000,0.0,5.778560e-07,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.160271,0.000000,0.0,0.0,1.385082,2.015524,1.835147,1.838054,2.572235,1.348832,0.0,0.0,0.0,1.905613,1.379344,0.000000,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
192002,1.506454e-06,0.007356,0.000000,0.001013,0.000000,0.692953,2.478337,0.000000,0.478590,0.000000,0.00000,0.0,1.300176e-06,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.369110,0.000000,0.0,0.0,0.369355,2.015524,0.244686,0.245074,0.342965,0.359689,0.0,0.0,0.0,0.238202,0.367825,0.000000,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
80789,1.673838e-06,0.007356,0.000000,0.000380,0.000000,0.623658,2.478337,0.000000,0.193829,0.000000,0.01525,0.0,1.444640e-06,0.000000,0.0,0.0,2.002085,0.0,0.0,0.0,0.0,0.0,0.0,0.101991,0.000000,0.0,0.0,0.646372,6.046572,0.244686,0.000000,0.000000,0.539533,0.0,0.0,0.0,0.476403,0.551738,0.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
79769,0.000000e+00,0.000000,0.000000,0.000127,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,8.667941e+00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.106848,0.000000,0.0,0.0,0.000000,2.015524,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,8.430444,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [67]:
from sklearn import model_selection
from sklearn import metrics
X = x_train.drop("label", axis = 1)
Y = x_train["label"]

In [68]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.linear_model import LogisticRegression
start_time = time.time()

pred_now, acc_lr, acc_cv_lr, lr = fit_algo(LogisticRegression(C = 0.1), X, Y, 10)

lr_time = (time.time() - start_time)

print("Accuracy: %s" % acc_lr)
print("Accuracy of CV: %s" % acc_cv_lr)
print("Execution time: %s" % lr_time)

Accuracy: 90.03
Accuracy of CV: 90.01
Execution time: 58.0999915599823


In [69]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
from sklearn.neighbors import KNeighborsClassifier
start_time = time.time()
pred_now, acc_knn, acc_cv_knn, knn = fit_algo(KNeighborsClassifier(n_neighbors = 3)
                                        , X, Y, 10)
knn_time = (time.time() - start_time)
print("Accuracy: %s" % acc_knn)
print("Accuracy of CV: %s" % acc_cv_knn)
print("Execution time: %s" % knn_time)

Accuracy: 95.63
Accuracy of CV: 91.22
Execution time: 1454.7340157032013


In [70]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
start_time = time.time()
pred_now, acc_dt, acc_cv_dt, dt = fit_algo(DecisionTreeClassifier(random_state = 1)
                                        , X, Y, 10)

dt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_dt)
print("Accuracy of CV: %s" % acc_cv_dt)
print("Execution time: %s" % dt_time)




Accuracy: 99.81
Accuracy of CV: 93.21
Execution time: 31.407108545303345


In [71]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Random Forest
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()
pred_now, acc_rf, acc_cv_rf, rf = fit_algo(RandomForestClassifier(n_estimators = 100)
                                        , X, Y, 10)

rf_time = (time.time() - start_time)

print("Accuracy: %s" % acc_rf)
print("Accuracy of CV: %s" % acc_cv_rf)
print("Execution time: %s" % rf_time)

Accuracy: 99.81
Accuracy of CV: 94.62
Execution time: 266.9949486255646


In [72]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gaussian Naive Bayes 
from sklearn.naive_bayes import GaussianNB
start_time = time.time()

pred_now, acc_gnb, acc_cv_gnb, gnb= fit_algo(GaussianNB()
                                        ,X,Y,5)

gnb_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gnb)
print("Accuracy of CV: %s" % acc_cv_gnb)
print("Execution time: %s" % gnb_time)

Accuracy: 50.48
Accuracy of CV: 50.47
Execution time: 3.896271228790283


In [73]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
start_time = time.time()

pred_now, acc_gbt, acc_cv_gbt, gbt= fit_algo(GradientBoostingClassifier()
                                        , X, Y, 10)

gbt_time = (time.time() - start_time)

print("Accuracy: %s" % acc_gbt)
print("Accuracy of CV: %s" % acc_cv_gbt)
print("Execution time: %s" % gbt_time)



Accuracy: 93.44
Accuracy of CV: 93.38
Execution time: 951.3844857215881


In [74]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
# Cross Validation Accuracy Comparison
algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'CV Accuracy %' : [acc_cv_lr, acc_cv_dt, acc_cv_rf, acc_cv_gnb, acc_cv_gbt, acc_cv_knn] })
acc_df = acc_df.sort_values(by='CV Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df


,Algorithm,CV Accuracy %
0,RandomForest,94.62
1,GBC,93.38
2,Decision Tree,93.21
3,KNN,91.22
4,Log. Reg.,90.01
5,Gaussian NB,50.47


In [ ]:
# Accuracy Comparison

algo_name = ['Log. Reg.', 'Decision Tree', 'RandomForest','Gaussian NB', 'GBC', 'KNN']
acc_df = pd.DataFrame({'Algorithm' : algo_name, 'Accuracy %' : [acc_lr, acc_dt, acc_rf, acc_gnb, acc_gbt, acc_knn] })
acc_df = acc_df.sort_values(by='Accuracy %', ascending = False)
acc_df = acc_df.reset_index(drop=True)
acc_df

In [ ]:
#Code copied from: Omar AlHory, Python, https://www.kaggle.com/omaralhory/ids-data-eda
fimp_rf = pd.DataFrame({'Feature' : X.columns, 'Importance' : (rf.feature_importances_).astype(float)})
fimp_rf = fimp_rf.sort_values(by='Importance', ascending=False)
fimp_rf.head(37)